In [2]:
function g(x,T,nmax;τ=0.01)
    a = 0im
    for n=-nmax:nmax
        ωn = pi*T*(2n+1)
        a += exp(im*ωn*τ)/(im*ωn-x)
    end
    return real(a*T)
end
xs = range(-1,1,length=10)


-1.0:0.2222222222222222:1.0

In [3]:
T = 0.1
nmax = 100000
for i=1:10
    println(xs[i],"\t",g(xs[i],T,nmax),"\t",1/(exp(xs[i]/T)+1))
end

-1.0	0.989498288411415	0.9999546021312976
-0.7777777777777778	0.9913302694409194	0.9995812333155556
-0.5555555555555556	0.990123551768255	0.9961489676440697
-0.3333333333333333	0.9618350473721775	0.9655548043337887
-0.1111111111111111	0.7509941347989852	0.7523361988609284
0.1111111111111111	0.24743253689074113	0.24766380113907163
0.3333333333333333	0.03405360537506296	0.03444519566621118
0.5555555555555556	0.0033658873349129608	0.003851032355930255
0.7777777777777778	-8.45627268651121e-5	0.0004187666844443735
1.0	-0.0004607450598311099	4.5397868702434395e-5


## 2.3 目的を達成するまで繰り返し続ける

In [4]:
function g2(x,T;τ=0.01,nmax=1000000,eps = 1e-8)
    n = 0
    ωn = pi*T*(2n+1)
    a = exp(im*ωn*τ)/(im*ωn-x)
    aold = a
    for n=1:nmax
        ωn = pi*T*(2n+1)
        a += exp(im*ωn*τ)/(im*ωn-x)
        ωn = pi*T*(2*(-n)+1)
        a += exp(im*ωn*τ)/(im*ωn-x)
        if abs(a-aold)/abs(aold) < eps
            println("converged at $n step")
            return T*a
        end
        aold = a
    end
    println("not converged in $nmax step")
    return T*a
end

g2 (generic function with 1 method)

for文だと必ず100万ステップ回すことになる。これだと余計に計算したりすることになる、もしくは、100万ステップ回したけど求めている精度にはならなかっったという風になるので、そう言う場合にはwhile文でかく。

In [5]:
function g2(x,T;τ=0.01,eps = 1e-8)
    n = 0
    ωn = pi*T*(2n+1)
    a = exp(im*ωn*τ)/(im*ωn-x)
    aold = 10*a
    while abs(a-aold)/abs(aold) > eps
        aold = a
        n += 1
        ωn = pi*T*(2n+1)
        a += exp(im*ωn*τ)/(im*ωn-x)
        ωn = pi*T*(2*(-n)+1)
        a += exp(im*ωn*τ)/(im*ωn-x)
    end
    println("converged in $n step")
    return T*a
end

g2 (generic function with 1 method)

In [6]:
g2(0,0.1;τ=0.001,eps=1e-15)

converged in 17845000 step


0.5000283892332869 + 8.918740923376492e-9im

## 2.4 行列とベクトルを扱う：配列

### 2.4.1

In [7]:
a=[1,2]

2-element Vector{Int64}:
 1
 2

In [8]:
A=[1 2
   3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [9]:
b=A*a

2-element Vector{Int64}:
  5
 11

In [10]:
a=(1/sqrt((2)))*[1,1]

2-element Vector{Float64}:
 0.7071067811865475
 0.7071067811865475

In [11]:
b=(1/sqrt(2))*[1,-1]

2-element Vector{Float64}:
  0.7071067811865475
 -0.7071067811865475

In [12]:
A=[0 -im
   im 0]

2×2 Matrix{Complex{Int64}}:
 0+0im  0-1im
 0+1im  0+0im

In [13]:
c=b'*A*a

0.0 - 0.9999999999999998im

In [14]:
c=(A'*b)'*a

0.0 - 0.9999999999999998im

### 2.4.2

In [15]:
a=[1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [16]:
B=[1 2 3 4
    5 6 7 8]

2×4 Matrix{Int64}:
 1  2  3  4
 5  6  7  8

In [17]:
B=[1 2 3 4;5 6 7 8]

2×4 Matrix{Int64}:
 1  2  3  4
 5  6  7  8

In [18]:
a=[1
2
3
4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [19]:
a=[1;2;3;4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [20]:
b=[1 2 3 4]

1×4 Matrix{Int64}:
 1  2  3  4

In [21]:
b*a

1-element Vector{Int64}:
 30

In [22]:
a*b

4×4 Matrix{Int64}:
 1  2   3   4
 2  4   6   8
 3  6   9  12
 4  8  12  16

In [23]:
B=zeros(3,4)

3×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [24]:
B[1,2]=3

3

In [25]:
B

3×4 Matrix{Float64}:
 0.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [26]:
function make_matrix(n)
    H=zeros(n,n)
    for i=1:n
        j=i+1
        j+=ifelse(j>n,-n,0)
        H[i,j]=-1
        j=i-1
        j+=ifelse(j<1,n,0)
        H[i,j]=-1
        H[i,i]=2
    end
    return H
end

make_matrix (generic function with 1 method)

In [27]:
make_matrix(10)

10×10 Matrix{Float64}:
  2.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0
 -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   2.0  -1.0
 -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   2.0

In [28]:
A =[1 2 3
    4 5 6
    7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [29]:
A[1:2,1:2]

2×2 Matrix{Int64}:
 1  2
 4  5

In [30]:
A[2:3,2:3]

2×2 Matrix{Int64}:
 5  6
 8  9

In [31]:
A[[1,3],[1,3]]

2×2 Matrix{Int64}:
 1  3
 7  9

In [32]:
A[:,1]

3-element Vector{Int64}:
 1
 4
 7

In [33]:
A[begin:end,1]

3-element Vector{Int64}:
 1
 4
 7

In [34]:
size(A)

(3, 3)

In [35]:
length(A)

9

### 2.4.3

In [36]:
using LinearAlgebra

In [37]:
a⋅b

30

In [38]:
a

4-element Vector{Int64}:
 1
 2
 3
 4

In [39]:
b

1×4 Matrix{Int64}:
 1  2  3  4

In [40]:
dot(a,b)

30

In [41]:
a=[3,2im]

2-element Vector{Complex{Int64}}:
 3 + 0im
 0 + 2im

In [42]:
dot(a,a)

13 + 0im

In [43]:
a'*a

13 + 0im

In [44]:
A=rand(ComplexF64,3,3)

3×3 Matrix{ComplexF64}:
 0.351737+0.195138im    0.522166+0.225626im  0.469654+0.915948im
 0.427378+0.00719696im  0.883247+0.2606im    0.679689+0.804769im
 0.368307+0.790815im    0.386641+0.659309im   0.21119+0.692897im

In [45]:
A'

3×3 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 0.351737-0.195138im  0.427378-0.00719696im  0.368307-0.790815im
 0.522166-0.225626im  0.883247-0.2606im      0.386641-0.659309im
 0.469654-0.915948im  0.679689-0.804769im     0.21119-0.692897im

In [46]:
transpose(A)

3×3 transpose(::Matrix{ComplexF64}) with eltype ComplexF64:
 0.351737+0.195138im  0.427378+0.00719696im  0.368307+0.790815im
 0.522166+0.225626im  0.883247+0.2606im      0.386641+0.659309im
 0.469654+0.915948im  0.679689+0.804769im     0.21119+0.692897im

In [47]:
a=rand(3)

3-element Vector{Float64}:
 0.937566546371274
 0.6125028028198298
 0.6378336860376124

In [48]:
cross(a,b)

LoadError: MethodError: no method matching cross(::Vector{Float64}, ::Matrix{Int64})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
[0mClosest candidates are:
[0m  cross(::AbstractVector, [91m::AbstractVector[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/generic.jl:310

In [ ]:
a

3-element Vector{Float64}:
 0.6724465881689338
 0.44486298586154527
 0.8281479813542175

In [ ]:
b

1×4 Matrix{Int64}:
 1  2  3  4

In [ ]:
cross(b,a)

LoadError: MethodError: no method matching cross(::Matrix{Int64}, ::Vector{Float64})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
[0mClosest candidates are:
[0m  cross([91m::AbstractVector[39m, ::AbstractVector) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/generic.jl:310

In [ ]:
b=[1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [ ]:
b=[1;2;3;4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [ ]:
b=[1 2 3 4]

1×4 Matrix{Int64}:
 1  2  3  4

In [ ]:
a

3-element Vector{Float64}:
 0.6724465881689338
 0.44486298586154527
 0.8281479813542175

In [ ]:
a=[1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [ ]:
b=[1 2 3 4]

1×4 Matrix{Int64}:
 1  2  3  4

In [ ]:
cross(b,a)

LoadError: MethodError: no method matching cross(::Matrix{Int64}, ::Vector{Int64})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
[0mClosest candidates are:
[0m  cross([91m::AbstractVector[39m, ::AbstractVector) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/generic.jl:310

In [ ]:
u=[1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [ ]:
v=[1,1,1]

3-element Vector{Int64}:
 1
 1
 1

In [ ]:
cross(u,v)

3-element Vector{Int64}:
 -1
  2
 -1

In [49]:
x=A\b

LoadError: DimensionMismatch: arguments must have the same number of rows